# 転移学習モデルの推論実行

https://github.com/pf-robotics/kachaka-transfer-learning に従い作成したonnxファイルを"predictor.onnx"として本ノートブックと同じディレクトリに配置して下さい。

In [ ]:
import asyncio

import kachaka_api
import numpy as np
import onnxruntime
from IPython.display import clear_output, display
from kachaka_api.util.vision import get_bbox_drawn_image_with_user_labels

In [ ]:
model = "predictor.onnx"
session = onnxruntime.InferenceSession(model)
input_name_list = [out.name for out in session.get_inputs()]
output_name_list = [out.name for out in session.get_outputs()]

class_num = 1
label_names = [f"{i}_class" for i in range(class_num)]
# up to 3 classes
colors = ["red", "blue", "green"]
scale_h = 720.0 / 224
scale_w = 1280.0 / 398
_SCORE_THRESHOLD = 0.5

client = kachaka_api.aio.KachakaApiClient()

In [ ]:
stream_i = client.front_camera_ros_compressed_image.stream()
stream_d = client.object_detection_features.stream()

while True:
    image, (features_header, features) = await asyncio.gather(
        anext(stream_i), anext(stream_d)
    )
    inputs = {
        feature.name.replace("_out", ""): np.array(
            feature.data, dtype=np.float32
        ).reshape(feature.shape)
        for feature in features
    }
    (bboxes, labels, scores) = session.run(output_name_list, inputs)
    img = get_bbox_drawn_image_with_user_labels(
        image,
        label_names,
        bboxes,
        labels,
        colors,
        scores,
        scale_h,
        scale_w,
        _SCORE_THRESHOLD,
    )
    clear_output(wait=True)
    display(img)